# Classification algorithms in practice

Source: The following parts were taken from [here](https://www.coursera.org/learn/neural-networks-deep-learning/notebook/zAgPl/logistic-regression-with-a-neural-network-mindset): 1, 2, 4

## 1 - Packages ##

First, let's run the cell below to import all the packages that you will need during this assignment. 
- [numpy](www.numpy.org) is the fundamental package for scientific computing with Python.
- [h5py](http://www.h5py.org) is a common package to interact with a dataset that is stored on an H5 file.
- [matplotlib](http://matplotlib.org) is a famous library to plot graphs in Python.
- [PIL](http://www.pythonware.com/products/pil/) and [scipy](https://www.scipy.org/) are used here to test your model with your own picture at the end.
- [scikit-learn](https://scikit-learn.org/stable/) "is an open source Python library that implements a range of machine learning, pre-processing, cross-validation and visualization algorithms using a unified interface." (text source: [here](https://www.geeksforgeeks.org/learning-model-building-scikit-learn-python-machine-learning-library/))

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import h5py
import scipy
import sklearn
from PIL import Image
from scipy import ndimage
from lr_utils import load_dataset

%matplotlib inline

## 2 - Overview of the Problem set ##

**Problem Statement**: You are given a dataset ("data.h5") containing:
    - a training set of m_train images labeled as cat (y=1) or non-cat (y=0)
    - a test set of m_test images labeled as cat or non-cat
    - each image is of shape (num_px, num_px, 3) where 3 is for the 3 channels (RGB). Thus, each image is square (height = num_px) and (width = num_px).

You will build a simple image-recognition algorithm that can correctly classify pictures as cat or non-cat.

Let's get more familiar with the dataset. Load the data by running the following code.

In [ ]:
# Loading the data (cat/non-cat)
train_set_x_orig, train_set_y_orig, test_set_x_orig, test_set_y_orig, classes = load_dataset()

We added "_orig" at the end of image datasets (train and test) because we are going to preprocess them. 

Each line of your train_set_x_orig and test_set_x_orig is an array representing an image. You can visualize an example by running the following code. Feel free also to change the `index` value and re-run to see other images. 

In [ ]:
# Example of a picture
index = 100 # you can change this
plt.imshow(train_set_x_orig[index])
print ("y = " + str(train_set_y_orig[:, index]) + ", it's a '" + classes[np.squeeze(train_set_y_orig[:, index])].decode("utf-8") +  "' picture.")

In [ ]:
train_set_x_orig.shape

Many software bugs in machine/deep learning come from having matrix/vector dimensions that don't fit. If you can keep your matrix/vector dimensions straight you will go a long way toward eliminating many bugs. 

**Exercise:** Find the values for:
    - m_train (number of training examples)
    - m_test (number of test examples)
    - num_px (= height = width of a training image)
Remember that `train_set_x_orig` is a numpy-array of shape (m_train, num_px, num_px, 3). For instance, you can access `m_train` by writing `train_set_x_orig.shape[0]`.

In [ ]:
m_train = train_set_x_orig.shape[0]
m_test = test_set_x_orig.shape[0]
num_px = train_set_x_orig.shape[1]

print ("Number of training examples: m_train = " + str(m_train))
print ("Number of testing examples: m_test = " + str(m_test))
print ("Height/Width of each image: num_px = " + str(num_px))
print ("Each image is of size: (" + str(num_px) + ", " + str(num_px) + ", 3)")
print ("train_set_x shape: " + str(train_set_x_orig.shape))
print ("train_set_y shape: " + str(train_set_y_orig.shape))
print ("test_set_x shape: " + str(test_set_x_orig.shape))
print ("test_set_y shape: " + str(test_set_y_orig.shape))

For convenience, you should now reshape images of shape (num_px, num_px, 3) in a numpy-array of shape (num_px $*$ num_px $*$ 3, 1). After this, our training (and test) dataset is a numpy-array where each column represents a flattened image. There should be m_train (respectively m_test) columns.

A trick when you want to flatten a matrix X of shape (a,b,c,d) to a matrix X_flatten of shape (b$*$c$*$d, a) is to use: 
```python
X_flatten = X.reshape(X.shape[0], -1).T      # X.T is the transpose of X
```

In [ ]:
train_set_x_flatten = train_set_x_orig.reshape(train_set_x_orig.shape[0],-1)
train_set_y = train_set_y_orig.reshape(train_set_y_orig.shape[1],)
test_set_x_flatten = test_set_x_orig.reshape(test_set_x_orig.shape[0],-1)
test_set_y = test_set_y_orig.reshape(test_set_y_orig.shape[1],)

print ("train_set_x_flatten shape: " + str(train_set_x_flatten.shape))
print ("train_set_y shape: " + str(train_set_y.shape))
print ("test_set_x_flatten shape: " + str(test_set_x_flatten.shape))
print ("test_set_y shape: " + str(test_set_y.shape))
print ("sanity check after reshaping: " + str(train_set_x_flatten[0:5,0]))

To represent color images, the red, green and blue channels (RGB) must be specified for each pixel, and so the pixel value is actually a vector of three numbers ranging from 0 to 255.

One common preprocessing step in machine learning is to center and standardize your dataset, meaning that you substract the mean of the whole numpy array from each example, and then divide each example by the standard deviation of the whole numpy array. But for picture datasets, it is simpler and more convenient and works almost as well to just divide every row of the dataset by 255 (the maximum value of a pixel channel).

<!-- During the training of your model, you're going to multiply weights and add biases to some initial inputs in order to observe neuron activations. Then you backpropogate with the gradients to train the model. But, it is extremely important for each feature to have a similar range such that our gradients don't explode. You will see that more in detail later in the lectures. !--> 

Let's standardize our dataset.

In [ ]:
train_set_x = train_set_x_flatten/255.
test_set_x = test_set_x_flatten/255.

<font color='blue'>
**What you need to remember:**

Common steps for pre-processing a new dataset are:
- Figure out the dimensions and shapes of the problem (m_train, m_test, num_px, ...)
- Reshape the datasets such that each example is now a vector of size (num_px \* num_px \* 3, 1)
- "Standardize" the data

## 3 - ML algorithms in practice ##

It's time to try some algorithms to distinguish cat images from non-cat images: decision tree, AdaBoost, Naive Bayes. Their implementation used below may not be exactly the same as we discussed them at seminars, but the main idea is the same.

In [ ]:
# for myDecisionTree
from sklearn import tree
def trainDecisionTree(X,Y):
    np.random.seed(0)
    model = tree.DecisionTreeClassifier(criterion="entropy")
    model.fit(X,Y)
    return model

In [ ]:
# for myAdaBoost
from sklearn import tree
from sklearn import ensemble
def trainAdaBoost(X,Y):
    model = ensemble.AdaBoostClassifier(tree.DecisionTreeClassifier(criterion="entropy",max_depth=1),
                         n_estimators=10)
    model.fit(X,Y)
    return model

In [ ]:
# for myNB
from sklearn import naive_bayes
def trainNB(X,Y):
    model = naive_bayes.GaussianNB()
    model.fit(X,Y)
    return model

**Exercise**: Compute the training accuracy for the decision tree:

**Expected result**: 1.0

<font color='blue'>
**Exercise 1**: Compute the leave-one-out cross-validation accuracy for the decision tree:

In [ ]:
# this may take a while...


**Exercise**: Compute the training accuracy for AdaBoost:

**Expected result**: 0.9090909090909091

<font color='blue'>
**Exercise 2**: Compute the leave-one-out cross-validation accuracy for AdaBoost:

In [ ]:
# this may take a while...


**Exercise**: Compute the training accuracy for Naive Bayes:

**Expected result**: 0.645933014354067

<font color='blue'>
**Exercise 3**: Compute the leave-one-out cross-validation accuracy for Naive Bayes:

In [ ]:
# this may take a while...


<font color='blue'>
**Exercise 4**: From those three algorithms, pick the best one in this context. Then train it on the whole training data. Then test it on the test data and report the accuracy on the test set.

In [ ]:
# Example of a picture + label given by a model.
model =  #... YOU MUST SET THIS
predictions = model.predict(test_set_x)
index = 5 # you can change this
plt.imshow(test_set_x.T[:,index].reshape((num_px, num_px, 3)))
print ("y = " + str(test_set_y[index]) + ", you predicted that it is a \"" + classes[predictions[index]].decode("utf-8") +  "\" picture.")

## 4 - Test with your own image ##

Congratulations on finishing this assignment. You can use your own image and see the output of your model. To do that:
    1. Click on "File" in the upper bar of this notebook, then click "Open".
    2. Add your image to this Jupyter Notebook's directory, in the "images" folder
    3. Change your image's name in the following code
    4. Run the code and check if the algorithm is right (1 = cat, 0 = non-cat)!

In [ ]:
model = # ... YOU MUST SET THIS
my_image = "fish.jpg"   # change this to the name of your image file 

# We preprocess the image to fit your algorithm.
fname = "images/" + my_image
image = np.array(ndimage.imread(fname, flatten=False))
image = image/255.
my_image = scipy.misc.imresize(image, size=(num_px,num_px)).reshape((1, num_px*num_px*3)).T
my_predicted_image = model.predict(my_image.T)

plt.imshow(image)
print("y = " + str(np.squeeze(my_predicted_image)) + ", your algorithm predicts a \"" + classes[int(np.squeeze(my_predicted_image)),].decode("utf-8") +  "\" picture.")